In [25]:
import tensorflow as tf
import os
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras import layers, models

For when doing GPU work

In [26]:
print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


# **UNET**

### **Setting up Data**

In [27]:
dataset_path = 'archive/Concrete' 

positive_images_path = os.path.join(dataset_path, 'Positive', 'Images')
positive_masks_path = os.path.join(dataset_path, 'Positive', 'Masks')
negative_images_path = os.path.join(dataset_path, 'Negative', 'Images')
negative_masks_path = os.path.join(dataset_path, 'Negative', 'Mask')

positive_images = [os.path.join(positive_images_path, img) for img in os.listdir(positive_images_path)]
positive_masks = [os.path.join(positive_masks_path, mask) for mask in os.listdir(positive_masks_path)]
negative_images = [os.path.join(negative_images_path, img) for img in os.listdir(negative_images_path)]
negative_masks = [os.path.join(negative_masks_path, mask) for mask in os.listdir(negative_masks_path)]

# Combine positive and negative examples
images = positive_images + negative_images
masks = positive_masks + negative_masks

data = list(zip(images, masks))
tf.random.set_seed(42)  # For reproducibility
tf.random.shuffle(data)
images, masks = zip(*data)

In [35]:
def load_image_mask(image_path, mask_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)  # Normalize to [0, 1]

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_jpeg(mask, channels=1)  # Assuming masks are single-channel
    mask = tf.image.convert_image_dtype(mask, tf.float32)  # Normalize to [0, 1]

    return image, mask

def get_image_mask_paths(dataset_path):
    images = []
    masks = []
    for label in ['Positive', 'Negative']:
        image_dir = os.path.join(dataset_path, label, 'Images')
        mask_dir = os.path.join(dataset_path, label, 'Masks')

        for image_file in os.listdir(image_dir):
            if image_file.endswith('.jpg'):
                images.append(os.path.join(image_dir, image_file))
                masks.append(os.path.join(mask_dir, image_file))

    return images, masks

dataset_path = 'archive/Concrete'   # Change to your dataset path
images, masks = get_image_mask_paths(dataset_path)

# Ensure lists are of the same length
assert len(images) == len(masks), "Images and masks must have the same length"

# Create dataset
dataset = tf.data.Dataset.from_tensor_slices((images, masks))
dataset = dataset.map(load_image_mask, num_parallel_calls=tf.data.AUTOTUNE)
dataset = dataset.batch(64).prefetch(tf.data.AUTOTUNE)  # Adjust batch size as needed

### **Model Buliding**

In [37]:
def unet(input_shape):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder
    c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    c1 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c1)
    p1 = tf.keras.layers.MaxPooling2D((2, 2))(c1)

    c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(p1)
    c2 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c2)
    p2 = tf.keras.layers.MaxPooling2D((2, 2))(c2)

    c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(p2)
    c3 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c3)
    p3 = tf.keras.layers.MaxPooling2D((2, 2))(c3)

    c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(p3)
    c4 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c4)
    p4 = tf.keras.layers.MaxPooling2D((2, 2))(c4)

    # Bottleneck
    c5 = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(p4)
    c5 = tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same')(c5)

    # Decoder
    u6 = tf.keras.layers.Conv2DTranspose(512, (2, 2), strides=(2, 2), padding='same')(c5)
    u6 = tf.keras.layers.concatenate([u6, c4])
    c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(u6)
    c6 = tf.keras.layers.Conv2D(512, (3, 3), activation='relu', padding='same')(c6)

    u7 = tf.keras.layers.Conv2DTranspose(256, (2, 2), strides=(2, 2), padding='same')(c6)
    u7 = tf.keras.layers.concatenate([u7, c3])
    c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(u7)
    c7 = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(c7)

    u8 = tf.keras.layers.Conv2DTranspose(128, (2, 2), strides=(2, 2), padding='same')(c7)
    u8 = tf.keras.layers.concatenate([u8, c2])
    c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(u8)
    c8 = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(c8)

    u9 = tf.keras.layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(c8)
    u9 = tf.keras.layers.concatenate([u9, c1])
    c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(u9)
    c9 = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(c9)

    outputs = tf.keras.layers.Conv2D(1, (1, 1), activation='sigmoid')(c9)  # For binary segmentation

    model = tf.keras.Model(inputs=[inputs], outputs=[outputs])
    return model

In [38]:
input_shape = (448, 448, 3)  # Adjust based on your input size
model = unet(input_shape)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(dataset, epochs=50, validation_split=0.2)  

Epoch 1/50


2024-10-30 23:37:28.736431: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.08GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.
2024-10-30 23:37:28.736491: W tensorflow/core/kernels/gpu_utils.cc:50] Failed to allocate memory for convolution redzone checking; skipping this check. This is benign and only means that we won't check cudnn for out-of-bounds reads and writes. This message will only be printed once.
2024-10-30 23:37:28.800949: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8907
2024-10-30 23:37:29.690584: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 3.22GiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were ava

RuntimeError: pybind11::error_already_set: MISMATCH of original and normalized active exception types: ORIGINAL ResourceExhaustedError REPLACED BY KeyboardInterrupt: <EMPTY MESSAGE>

At:
  /home/eugene/miniconda3/lib/python3.11/site-packages/tensorflow/python/framework/errors_impl.py(377): __init__
  /home/eugene/miniconda3/lib/python3.11/site-packages/tensorflow/python/eager/execute.py(53): quick_execute
  /home/eugene/miniconda3/lib/python3.11/site-packages/tensorflow/python/eager/context.py(1457): call_function
  /home/eugene/miniconda3/lib/python3.11/site-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py(196): __call__
  /home/eugene/miniconda3/lib/python3.11/site-packages/tensorflow/python/eager/polymorphic_function/monomorphic_function.py(1349): _call_flat
  /home/eugene/miniconda3/lib/python3.11/site-packages/tensorflow/python/eager/polymorphic_function/tracing_compiler.py(148): __call__
  /home/eugene/miniconda3/lib/python3.11/site-packages/tensorflow/pyt

### **Visualising Results on a sample image**

In [ ]:
import matplotlib.pyplot as plt

# Function to visualize results
def visualize_results(image, true_mask, predicted_mask):
    plt.figure(figsize=(12, 4))
    
    # Original image
    plt.subplot(1, 3, 1)
    plt.imshow(image)
    plt.title('Original Image')
    plt.axis('off')

    # True mask
    plt.subplot(1, 3, 2)
    plt.imshow(true_mask.squeeze(), cmap='gray')
    plt.title('True Mask')
    plt.axis('off')

    # Predicted mask
    plt.subplot(1, 3, 3)
    plt.imshow(predicted_mask.squeeze(), cmap='gray')
    plt.title('Predicted Mask')
    plt.axis('off')

    plt.show()

# Select a sample image and its true mask
sample_index = 0  # Change this index to visualize different samples
sample_image = images[sample_index].numpy()  # Convert tensor to numpy array
sample_true_mask = masks[sample_index].numpy()  # Convert tensor to numpy array

# Reshape for the model prediction (adding batch dimension)
sample_image_input = np.expand_dims(sample_image, axis=0)

# Predict the mask
sample_predicted_mask = model.predict(sample_image_input)[0]  # Get the first image's prediction

# Threshold the predicted mask for binary segmentation
sample_predicted_mask = (sample_predicted_mask > 0.5).astype(np.float32)

# Visualize the results
visualize_results(sample_image, sample_true_mask, sample_predicted_mask)